# Angels - Tale

This is a colab demo notebook using the open source project [CorentinJ/Real-Time-Voice-Cloning](https://github.com/CorentinJ/Real-Time-Voice-Cloning)
to clone a voice.

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).


Original issue: https://github.com/tugstugi/dl-colab-notebooks/issues/18

## Setup CorentinJ/Real-Time-Voice-Cloning

In [1]:
#@title Setup CorentinJ/Real-Time-Voice-Cloning

#@markdown * clone the project
#@markdown * download pretrained models
#@markdown * initialize the voice cloning models

%tensorflow_version 1.x
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/HEC2018/Real-Time-Voice-Cloning.git'
project_name = splitext(basename(git_repo_url))[0]
final_wav = []
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  # install dependencies
  !cd {project_name} && pip install -q -r requirements.txt
  !pip install -q gdown
  !apt-get install -qq libportaudio2
  !pip install -q https://github.com/HEC2018/dl-colab-notebooks/archive/colab_utils.zip
  !pip install essentia
  !pip install pip --upgrade
  !pip uninstall --yes matplotlib
  !pip install matplotlib==2.2.3
  !pip install anvil-uplink
  !pip install dropbox
  !git clone https://github.com/HEC2018/Angels-Tale.git
  %cd Angels-Tale/
  !rm vox-cpk.tar
  !wget https://github.com/HEC2018/Angels-Tale/blob/master/vox-cpk.tar?raw=true
  !mv vox-cpk.tar\?raw\=true vox-cpk.tar
  %cd ..
  !git clone https://github.com/HEC2018/first-order-model.git
  # download pretrained model
  !cd {project_name} && wget https://github.com/HEC2018/Angels-Tale/blob/main/pretrained.zip?raw=true && mv pretrained.zip?raw=true pretrained.zip && unzip pretrained.zip

import sys
sys.path.append(project_name)
from essentia.standard import MonoWriter
from IPython.display import display, Audio, clear_output
from IPython.utils import io
import ipywidgets as widgets
import numpy as np
from dl_colab_notebooks.audio import record_audio, upload_audio
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
from google.colab import files
from io import BytesIO
from PIL import Image
from skimage import img_as_ubyte
import imageio
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
from base64 import b64encode
import anvil.server
import dropbox
import uuid


anvil.server.connect("5ZDHHJNHEF6TLTZMXUUX662U-2XLKAA7XUZFON6SX")

encoder.load_model(project_name / Path("encoder/saved_models/pretrained.pt"))
synthesizer = Synthesizer(project_name / Path("synthesizer/saved_models/logs-pretrained/taco_pretrained"))
vocoder.load_model(project_name / Path("vocoder/saved_models/pretrained/pretrained.pt"))



TensorFlow 1.x selected.
     |████████████████████████████████| 412.3MB 38kB/s 
     |████████████████████████████████| 686kB 53.5MB/s 
     |████████████████████████████████| 184kB 51.9MB/s 
     |████████████████████████████████| 11.6MB 47.0MB/s 
     |████████████████████████████████| 14.5MB 250kB/s 
     |████████████████████████████████| 245kB 57.5MB/s 
     |████████████████████████████████| 68.3MB 90kB/s 
     |████████████████████████████████| 204kB 53.8MB/s 
     |████████████████████████████████| 61kB 10.0MB/s 
     |████████████████████████████████| 286kB 63.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 145483 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1_amd64.d

     |████████████████████████████████| 58 kB 6.7 MB/s 
     |████████████████████████████████| 51 kB 242 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45215 sha256=5f7dbe14c81f7aa1d28ca4afeabed6dc4defe03e610a696e0de1c0e7a86d0045
  Stored in directory: /root/.cache/pip/wheels/e3/a6/ee/ea9de3c767ffa503e9bdb32da1f69826b62b3c89c87a918d0a
Successfully built ws4py


     |████████████████████████████████| 551 kB 14.0 MB/s 
     |████████████████████████████████| 160 kB 73.3 MB/s 
     |████████████████████████████████| 49 kB 9.0 MB/s 
Cloning into 'Angels-Tale'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 17 (delta 1), reused 12 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), done.
/content/Angels-Tale
--2021-01-16 07:01:50--  https://github.com/HEC2018/Angels-Tale/blob/master/vox-cpk.tar?raw=true
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/HEC2018/Angels-Tale/blob/main/vox-cpk.tar [following]
--2021-01-16 07:01:50--  https://github.com/HEC2018/Angels-Tale/blob/main/vox-cpk.tar
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 200 OK
Length: uns

Real-Time-Voice-Cloning/encoder/audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at Real-Time-Voice-Cloning/vocoder/saved_models/pretrained/pretrained.pt


In [ ]:
#@title Record or Upload Voice, and Synthesize the voice
#@markdown * Either record audio from microphone or upload audio from file (.mp3 or .wav) 

def create_voice(id):
  SAMPLE_RATE = 22050
  record_or_upload = "Upload (.mp3)" #@param ["Record", "Upload (.mp3)"]
  record_seconds =   10#@param {type:"number", min:1, max:10, step:1}

  embedding = None
  def _compute_embedding(audio):
    display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
    global embedding
    embedding = None
    embedding = encoder.embed_utterance(encoder.preprocess_wav(audio, SAMPLE_RATE))
  def _record_audio(b):
    clear_output()
    audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
    _compute_embedding(audio)
  def _upload_audio(b):
    clear_output()
    audio = upload_audio(sample_rate=SAMPLE_RATE)
    _compute_embedding(audio)

  if record_or_upload == "Record":
    button = widgets.Button(description="Record Your Voice")
    button.on_click(_record_audio)
    display(button)
  else:
    button = widgets.Button(description="Upload Voice File")
    button.on_click(_upload_audio)
    _upload_audio("")

  text = "A heap is a tree in which every node has a numerical value"  
  def synthesize(embed, text):
    print("Synthesizing new audio...")
    #with io.capture_output() as captured:
    specs = synthesizer.synthesize_spectrograms([text], [embed])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    clear_output()
    display(Audio(generated_wav, rate=synthesizer.sample_rate, autoplay=True))
    return generated_wav
  if embedding is None:
    print("first record a voice or upload a voice file!")
  else:
    MonoWriter(filename='user.wav',sampleRate=synthesizer.sample_rate)(synthesize(embedding, text))


Saving user.mp3 to user.mp3


In [ ]:
#@title Upload Images
#@markdown * Either record audio from microphone or upload audio from file (.mp3 or .wav) 


upload = "Upload (user.jpg)" #@param ["Upload (user.jpg)"]
#record_seconds =   10#@param {type:"number", min:1, max:10, step:1}

embedding = None

def _upload_pic(b):
  clear_output()
  uploaded = files.upload()
  im = Image.open(BytesIO(uploaded['user.jpg']))

button = widgets.Button(description="Upload Square image File")
button.on_click(_upload_pic)
_upload_pic("")

Saving user.jpg to user.jpg


In [ ]:
@anvil.server.callable
def create_story(id):
  %cd first-order-model
  %ls
  %ls ..
  warnings.filterwarnings("ignore")
  video_name = "generated_" + str(id) + ".mp4"

  source_image = imageio.imread('../user.jpg')
  reader = imageio.get_reader('../Angels-Tale/mother_' + str(id) + '.mp4')

  #Resize image and video to 256x256

  source_image = resize(source_image, (256, 256))[..., :3]

  fps = reader.get_meta_data()['fps']
  driving_video = []
  try:
      for im in reader:
          driving_video.append(im)
  except RuntimeError:
      pass
  reader.close()

  driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

  """def display(source, driving, generated=None):
      fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

      ims = []
      for i in range(len(driving)):
          cols = [source]
          cols.append(driving[i])
          if generated is not None:
              cols.append(generated[i])
          im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
          plt.axis('off')
          ims.append([im])

      ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
      plt.close()
      return ani
  """ 

  #HTML(display(source_image, driving_video).to_html5_video())

  # Create the video
  from demo import load_checkpoints
  from demo import make_animation
  generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                              checkpoint_path='../Angels-Tale/vox-cpk.tar')
  predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

  #save resulting video
  imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
  #video can be downloaded from /content folder
  %cd ..
  # Merging video and voice 
  !ffmpeg -y -i generated.mp4 -filter:a "atempo=2,atempo=1.1" generated_2.mp4
  !ffmpeg -y -i user.wav -filter:a "atempo=0.8" user_2.wav
  !ffmpeg -y -i generated_2.mp4 -i user_2.wav -c:v copy -c:a aac generated.mp4
  !mv generated.mp4 {video_name}.mp4
#return output.mp4

first-order-model/             Real-Time-Voice-Cloning/  user.jpg  user.wav
Interview_Practice_Questions/  sample_data/              user.mp3


"def display(source, driving, generated=None):\n    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))\n\n    ims = []\n    for i in range(len(driving)):\n        cols = [source]\n        cols.append(driving[i])\n        if generated is not None:\n            cols.append(generated[i])\n        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)\n        plt.axis('off')\n        ims.append([im])\n\n    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)\n    plt.close()\n    return ani\n"

In [ ]:
@anvil.server.callable
def retrieve(pic, voi):
  #
  # Extract the pic and voi file here
  # pictures have name as user.jpg
  # voice file has name as user.mp3
  #
  for i in range(1:6):
    create_story(i)

  videolinks = []
  def upload_video(id):
    video_gen_path = "generated_" + str(id) + "".mp4"

    class TransferData:
        def __init__(self, access_token):
            self.access_token = access_token

        def upload_file(self, file_from, file_to):
            """upload a file to Dropbox using API v2
            """
            dbx = dropbox.Dropbox(self.access_token)

            with open(file_from, 'rb') as f:
                dbx.files_upload(f.read(), file_to)

        def get_link(self, filePath):

          dbx = dropbox.Dropbox(self.access_token)
          link = dbx.files_get_temporary_link(filePath)

          return link

    def upload_get(file_from):
        access_token = "sl.ApYzt6QwqACUXKvh3hcsAB2EBJ87NnbsLfVqP9ts6j_AJcb5QOYj_XJJvLt1EkBhjYtePjUmEpKQPM0ZBs_CvhSurU1kKARF2NAPwuVi5cZZ-yeRYgxpUiKPKLZ-n_fOeDTObYon51k"
        transferData = TransferData(access_token)

        temp = str(uuid.uuid4())
        temp = "".join(temp.split("-"))
        file_to = "/" + temp + file_from  # The full path to upload the file to, including the file name

        # API v2
        transferData.upload_file(file_from, file_to)

        return transferData.get_link(file_to).link

    video_link = upload_get(video_gen_path)
  for i in range(1:6):
    videolinks.append(upload_video(i))
  return videolinks

In [ ]:
# Show the video
mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:

@anvil.server.callable
def test_function(name):
  print("Hello from the uplink, %s!" % name)

anvil.server.wait_forever()